# Readout data from database

**Note!** This notebook cannot be run due to its dependence on the underlying Datajoint database. It is only meant to showcase how the data was extracted from the data base.

In [1]:
# DATAJOINT
import datajoint as dj
dj.config.load('./utils/dj_mysql_conf.json')
c = dj.conn()

from schemata.cell import *
from schemata.density import *
from schemata.morphometry import *
from schemata.features import *
from schemata.classification import *

import pandas as pd

# PLOTTING

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Connecting slaturnus@172.25.240.205:3306


/gpfs01/berens/user/slaturnus/.local/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/gpfs01/berens/user/slaturnus/.local/lib/python3.5/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
path = "../../../publications/morphology-benchmark/"

In [3]:
# Features

key = dict(reduction_id=0)
for s in (FeatureMap()& key & "part_id=1" & "statistic_id < 110").fetch('statistic_id'):
    for p in [(1,'full'),(2,'axon'),(3,'dendrite')]:
        key['statistic_id'] = s
        key['part_id'] = p[0]
        df = FeatureMap().get_as_dataframe(key, "type != 'pyr' and part_id <4 and ds_id<6 and ds_id != 2")
        df.T.to_csv(path +"data/features/statistic_id_%i_%s.csv"%(s,p[1]), index=True)

In [4]:
# pairwise classification performance
df = pd.DataFrame((PairwiseClassificationImbalanced().CVRun() & "statistic_id < 110" & "ds_id !=2 and ds_id < 6").fetch(as_dict=True))
del df['training_indices']
del df['test_indices']
del df['reduction_id']
df.to_csv(path + "data/classification/pairwise_classification_performances.csv", index=False)

In [5]:
# multi class performance
df_multi = pd.DataFrame((MultiClassClassificationImbalanced.CVRun() & "statistic_id < 110" & "ds_id !=2 and ds_id < 6").fetch(as_dict=True))

del df_multi['training_indices']
del df_multi['test_indices']
del df_multi['reduction_id']

df_multi.to_csv(path + "data/classification/multiclass_classification_performances.csv", index=False)

In [6]:
# statistics
df_dm = pd.DataFrame(Statistic().DensityMap().fetch(as_dict=True))
df_graph = pd.DataFrame(Statistic().Graph().fetch(as_dict=True))
df_morphometrics = pd.DataFrame(Statistic().Morphometry().fetch(as_dict=True))
df_morph_dist = pd.DataFrame(Statistic().MorphometryDistribution().fetch(as_dict=True))
df_combined = pd.DataFrame(Statistic().Combined().fetch(as_dict=True))
df_persistence = pd.DataFrame(Statistic().Persistence().fetch(as_dict=True))

df_statistics = pd.DataFrame()

for d in [df_dm, df_graph,df_morphometrics, df_morph_dist, df_persistence, df_combined]:
    df_statistics = pd.concat((df_statistics, d[['statistic_id', 'statistic_name']]))

df_statistics.to_csv(path + "data/statistic_descriptions.csv", index=False)

In [73]:
# store the used reconstructions
ds_ids = [1,3,4,5]
selected_cells = []
for ds_id in ds_ids:
    cell_types = np.unique(((Cell() & "ds_id = %s"%str(ds_id)) - Cell().Ignore()).fetch('type'))

    for c_t in cell_types:
        if c_t not in ['pyr', 'IPC', 'UTPC', 'TPC', 'CBC8', 'CBC9']:
            selected_cells.append(list(((Cell() - Cell().Ignore())& "ds_id = %s"%str(ds_id) & "type='%s'"%c_t).fetch()[0]))
        

cells = pd.DataFrame(selected_cells, columns=['ds_id', 'c_num', 'type'])

# rename the cell type to be consistent with L4 paper
cells.loc[(cells['ds_id'] == 5) & (cells['type'] == 'BC'),'type'] = 'LBC'
cells.loc[(cells['ds_id'] == 5) & (cells['type'] == 'SC'),'type'] = 'SBC'
cells.loc[(cells['ds_id'] == 5) & (cells['type'] == 'HEC'),'type'] = 'HBC'
cells.to_csv(path + "/data/reconstructions/selected_cells.csv")

# get the number of cells

cell_numbers = pd.DataFrame((Cell() - Cell().Ignore()).fetch(as_dict=True))
cell_numbers = cell_numbers.groupby(['ds_id', 'type']).count().reset_index()

cell_numbers.loc[(cell_numbers['ds_id'] == 5) & (cell_numbers['type'] == 'BC'),'type'] = 'LBC'
cell_numbers.loc[(cell_numbers['ds_id'] == 5) & (cell_numbers['type'] == 'SC'),'type'] = 'SBC'
cell_numbers.loc[(cell_numbers['ds_id'] == 5) & (cell_numbers['type'] == 'HEC'),'type'] = 'HBC'

 cell_numbers[(cell_numbers['ds_id'] < 6) & (cell_numbers['ds_id'] != 2) & (cell_numbers['type'] != 'pyr')].to_csv("./data/reconstructions/cell_numbers.csv")

In [85]:
for _, item in cells[(cells['ds_id'] < 6) & (cells['ds_id'] != 2)].iterrows():
    neuron = Neuron().get_tree({'c_num': item['c_num'], 'ds_id': item['ds_id'], 'reduction_id':0})
    neuron.write_to_swc("ds_%i_cell_%i"%(item['ds_id'], item['c_num']), path=path + 'data/reconstructions/')